# DBSCAN clustering to reduce spatial data set size

This notebook reduces the size of a spatial data set by clustering with DBSCAN. More info: http://geoffboeing.com/2014/08/visualizing-summer-travels/

You might also be interested in [this notebook](https://github.com/gboeing/data-visualization/blob/master/location-history/google-location-history-cluster.ipynb) that uses this technique to cluster 1.2 million spatial data points and [this write-up](http://geoffboeing.com/2016/06/mapping-everywhere-ever-been/) of that project. Also see [here](https://en.wikipedia.org/wiki/Earth_radius#Mean_radius) for the number of kilometers in one radian.

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import folium
%matplotlib inline

In [2]:
# define the number of kilometers in one radian
kms_per_radian = 6371.0088

In [3]:
# load the data set
df = pd.read_csv('data/summer-travel-gps-full.csv', encoding='utf-8')
df.head()

,lat,lon,date,city,country
0,51.481292,-0.451011,05/14/2014 09:07,West Drayton,United Kingdom
1,51.474005,-0.450999,05/14/2014 09:22,Hounslow,United Kingdom
2,51.478199,-0.446081,05/14/2014 10:51,Hounslow,United Kingdom
3,51.478199,-0.446081,05/14/2014 11:24,Hounslow,United Kingdom
4,51.474146,-0.451562,05/14/2014 11:38,Hounslow,United Kingdom


The scikit-learn DBSCAN haversine distance metric requires data in the form of [latitude, longitude] and both inputs and outputs are in units of radians.

### Compute DBSCAN

  - `eps` is the physical distance from each point that forms its neighborhood
  - `min_samples` is the min cluster size, otherwise it's noise - set to 1 so we get no noise
  
Extract the lat, lon columns into a numpy matrix of coordinates, then convert to radians when you call `fit`, for use by scikit-learn's haversine metric.

In [7]:
# represent points consistently as (lat, lon)
coords = df[['lat', 'lon']].values

# define epsilon as 1.5 kilometers, converted to radians for use by haversine
epsilon = 1.5 / kms_per_radian

In [9]:
start_time = time.time()
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_

# get the number of clusters
num_clusters = len(set(cluster_labels))

# all done, print the outcome
message = 'Clustered {:,} points down to {:,} clusters, for {:.1f}% compression in {:,.2f} seconds'
print(message.format(len(df), num_clusters, 100*(1 - float(num_clusters) / len(df)), time.time()-start_time))
print('Silhouette coefficient: {:0.03f}'.format(metrics.silhouette_score(coords, cluster_labels)))

Clustered 1,759 points down to 138 clusters, for 92.2% compression in 0.39 seconds
Silhouette coefficient: 0.652


In [10]:
# turn the clusters in to a pandas series, where each element is a cluster of points
clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])

### Find the point in each cluster that is closest to its centroid

DBSCAN clusters may be non-convex. This technique just returns one representative point from each cluster. First get the lat,lon coordinates of the cluster's centroid (shapely represents the *first* coordinate in the tuple as `x` and the *second* as `y`, so lat is `x` and lon is `y` here). Then find the member of the cluster with the smallest great circle distance to the centroid.

In [54]:
def get_centermost_point(cluster):
    count = len(cluster)
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)    
    ## To find the furthest point
#     centermost_point = max(cluster, key=lambda point: great_circle(point, centroid).m)
#     return tuple(centermost_point)
    
    # to get radius in meter
    radius_m = max(great_circle(point, centroid).m for point in cluster)
    print((centroid, radius_m, count))
    return (centroid, radius_m, count)
    
cent_rad = clusters.map(get_centermost_point)

((51.47795081428571, -0.44683728571428577), 534.8953045802483, 7)
((38.774392125000006, -9.130013075), 1048.8504768404268, 4)
((38.74298710000001, -9.1477802), 0.0, 1)
((38.71229583533834, -9.13658587518797), 2361.5621851269652, 133)
((38.69550269166667, -9.208059566666666), 761.8095536447532, 12)
((38.7025124, -9.1805038), 0.0, 1)
((38.73966218, -9.1671093), 980.0466770104817, 5)
((38.7616021, -9.2450257), 0.0, 1)
((38.797558200000005, -9.3409997), 0.0, 1)
((38.79529771, -9.38444829), 2255.644407007648, 10)
((38.801171200000006, -9.4251031), 0.0, 1)
((38.793071000000005, -9.2858255), 0.0, 1)
((38.816610700000005, -9.408499800000001), 0.0, 1)
((38.7686867, -9.2999259), 0.0, 1)
((38.7472838, -9.213838800000001), 0.0, 1)
((39.1183833, -8.913737800000002), 0.0, 1)
((39.27287570000001, -8.7121869), 0.0, 1)
((39.476521999999996, -8.634345300000001), 0.0, 1)
((39.6297627, -8.6881566), 607.783839169505, 2)
((39.841085799999995, -8.718608), 0.0, 1)
((40.0204289, -8.5944898), 0.0, 1)
((40.21086

In [90]:
# unzip the list of centermost points (lat, lon) tuples into separate lat and lon lists
coordinates, rad,count = zip(*cent_rad)
lats, lons = zip(*coordinates)

# from these lats/lons create a new df of one representative point for each cluster
rep_points = pd.DataFrame({ 'lat':lats,'lon':lons, 'rad':rad, 'count':count})
rep_points['density'] = rep_points['count']/(rep_points['rad'] + 1)
rep_points.describe()

,lat,lon,rad,count,density
count,138.000000,138.000000,138.000000,138.000000,138.000000
mean,45.552934,10.190861,338.672630,12.746377,0.733624
std,4.681410,10.899437,812.878421,57.351148,0.476915
min,37.921659,-9.425103,0.000000,1.000000,0.002749
25%,40.894859,7.877534,0.000000,1.000000,0.069782
50%,48.368578,13.097103,0.000000,1.000000,1.000000
75%,49.880714,19.076143,189.946491,3.000000,1.000000
max,51.477951,29.009547,4407.068084,631.000000,2.000000


In [59]:
## pull row from original data set where lat/lon match the lat/lon of each row of representative points
## that way we get the full details like city, country, and date from the original dataframe

# rs = rep_points.apply(lambda row: df[(df['lat']==row['lat']) & (df['lon']==row['lon'])].iloc[0], axis=1)
# rs.to_csv('data/summer-travel-gps-dbscan.csv', encoding='utf-8')
# rs.tail()

In [60]:
## plot the final reduced set of coordinate points vs the original full set

# fig, ax = plt.subplots(figsize=[10, 6])
# rs_scatter = ax.scatter(rs['lon'], rs['lat'], c='#99cc99', edgecolor='None', alpha=0.7, s=120)
# df_scatter = ax.scatter(df['lon'], df['lat'], c='k', alpha=0.9, s=3)
# ax.set_title('Full data set vs DBSCAN reduced set')
# ax.set_xlabel('Longitude')
# ax.set_ylabel('Latitude')
# ax.legend([df_scatter, rs_scatter], ['Full set', 'Reduced set'], loc='upper right')
# plt.show()

In [72]:
center = [rep_points.lat.mean(), rep_points.lon.mean()]

In [101]:
from folium import plugins
import branca.colormap as cm

colormap = cm.linear.Blues_05.scale(vmin=rep_points.density.min(),vmax=rep_points.density.max())

In [102]:
map1 = folium.Map(location= center, zoom_start=6)
incidents = plugins.MarkerCluster().add_to(map1)

for lat, lng, rad, c in zip(rep_points.lat,rep_points.lon, rep_points.rad,rep_points.density):
    folium.Circle(
        location=[lat, lng],
        radius= rad,
        fill=True,
        color=colormap(c)
    ).add_to(incidents)


# folium.Circle([lat, lon],
#                     radius=40
#                    ).add_to(map)

map1